In [ ]:
#Inizio parte sui classificatori

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc, RocCurveDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

Creo il mio DataFrame con i dati necessari sui consumi (totali?), poi specializzo nell'area urbana di Trento.

In [ ]:
df_l = pd.read_csv('dataframe/dati_orari_medie_provincia_Trento.csv')
#POSSIAMO USARE ANCHE I DATI ORARI TOTALI AL POSTO DELLE MEDIE

In [ ]:
df_l = df_l[df_l['COMUNE']=='TRENTO']

Per il preprocessing dei dati, uso come riferimento del filtro la classe di consumo 'alta'

In [ ]:
filter = [df_l['fascia_oraria']=='8-12',df_l['fascia_oraria']=='12-14',df_l['fascia_oraria']=='14-19',(df_l['fascia_oraria']!='19-00')&(df_l['fascia_oraria']!='00-08')]

df = df_l[filter[2]]
target_class = 'alto'

Qui ci troviamo di fronte a due possibili scelte di insiemi di training e test. Tenendo conto della sequenza temporale, e quindi volendo analizzare i consumi futuri, possiamo usare come training il mese di novembre e la prima metà di dicembre, per fare il test sulla seconda metà di dicembre. Siccome tale scelta produce discrepanze viziate dalle vacanze natalizie, la seconda scelta è una divisione casuale (80%-20%) dei giorni, in modo da avere dei risultati più mediati, in positivo e/o in negativo.

Qui riporto la prima scelta, quella basata su criteri temporali.

In [ ]:
# Divido il dataset in training e test (prima e dopo il 15 dicembre)
X_train = df[df['data'] < '2013-12-15'].drop(columns = {'consumo','data','corrente_pesata','COMUNE'})
# Creo un array con gli identificativi delle celle del training
cell_train = X_train['cellId'].unique()
X_test = df.query("data >= '2013-12-15' and cellId in @cell_train").drop(columns = {'consumo', 'data', 'corrente_pesata', 'COMUNE'})
y_train = df[df['data'] < '2013-12-15']['consumo']
y_train = (y_train == target_class).astype(int)
y_test = df.query("data >= '2013-12-15' and cellId in @cell_train")['consumo']
y_test = (y_test == target_class).astype(int)

Adesso riporto la seconda opzione, quella di divisione casuale.

In [ ]:
X = df.drop(columns= {'consumo','data','corrente_pesata','COMUNE'})
y = df['consumo']
y_bin = (y == target_class).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y_bin, test_size=0.2, random_state=24)

Predispongo la pipeline per un lavoro preliminare sui dati. Per quanto riguarda i dati mancanti, abbiamo agito in due modi al variare del tipo di feature (numerica o di categoria). Per la prima, sostituisco il dato problematico con una media e poi normalizzo tutte le features. Per la seconda, invece, sostituisco con valore 'missing', in quanto pensiamo che questa nuova categoria non influenzi particolarmente l'analisi complessiva.

In [ ]:
numeric_features = ['temp','prec','tweet','pioggia','mod_wind','ang_wind']
categorical_features = ['cellId','fascia_oraria','day_week']

#Per le features numeriche, uso la media in caso di dati mancanti, poi normalizzo tutto
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

#Per le features di categoria, sostituisco le celle mancanti con 'missing' values, mentre ignoro successivamente i valori non visti durante il training
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output = False))
])
       
#Unisco le due pipelines precedenti in un'unica struttura
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [ ]:
# Creazione della pipeline (sta parte sto ancora capendo meglio asp)
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

model_rf = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', RandomForestClassifier(random_state=24))])

model_lr = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', LogisticRegression(random_state=24))])